## Sport calendar Optimization:
    
We have to setup the ***best*** calendar for an international sport federation with the following rules:
    1. there are 9 Teams coming from 9 locations all over the world
    2. the tournament is divided in 4 consecutive Weekends.
    3. Each round consists in 3 Groups of 3 Teams, and each Group takes place in one of the home-location of the 3 partecipants
    4. In the tournament, each team will play against all the other teams excatly once in the 4 Weekends (two teams will attend the same group excatly once)
    5. Between Weekends, all Teams will fly directy from the current location to the next one (no back home)
    6. Each Team must host at least one group
    7. For each potential group location and for each turn, a forecast of potential spectators is provided
    
    x. Find the best calendar which, at the same time, minimizes the total travelled distance by the players and maximizes the forecasted spectators



### Sets


\begin{align}
& \text{Locations}  &L &  & & \forall l \in L \\
& \text{Teams}  &T &  & &\forall t \in T \\
& \text{Weekend} &W  &  & &\forall w \in W\\
&\text{Groups}  &G  &  & &\forall g \in G\\  
\end{align}
### Parameters
\begin{align}
& \text{Distance matrix} & D_{ll'} & &  & & \forall l,l' \in L\\
& \text{Expected fans per Weekend in each location} & F_{lw}  & &  & & \forall l \in L, w \in W\\
& \text{Team <> Location map} & M_{tl}  & &  & & \forall t \in T, l \in L\\
&  \text{Weight parameter} & P    & & & &
\end{align}


### Variables
\begin{align}
& \text{Team playing in pool (g,w)} & x_{tgw} & \in \{0,1\} &  & & \forall t \in T, g \in G,w \in W & \tag{V1} \\
& \text{Location hosting pool (g,w)}  & y_{lgw} & \in \{0,1\} & & & \forall l \in L, g \in G,w \in W & \tag{V2} \\
& & & & & & &\\
& \text{Flight from $l$ to $l'$ between $w$ and $w+1$} & z_{ll'w} & \in \{0,1\}&  & &\forall l,l' \in L, w \in w & \tag{V3} \\
          & & & = \sum_{g} y_{lgw} * \sum_{g'} y_{l'g'w+1}  & & &  & & &  & !Achtung!\triangle
\end{align}



### Objective

\begin{align}
& \text{ } & \text{Total Travelled distance} & & E_T  & = & \sum_{ll'w} D_{ll'} * z_{ll'w} &  \\
& \text{ } & \text{Total Attending Fans} &  & E_F  & = & \sum_{glw} F_{lw} * y_{lgw} & \\
 & & & &  \min_{zy}   E_T - P * E_F & \tag{obj}
\end{align}

### Constraints

\begin{align}
& \text{Each player plays in a group each turn} & \sum_{g} x_{tgw} = 1 & & \forall t \in T, w \in W \tag{C1}\\
& \text{Each group consists of 3 teams each Turn} & \sum_{p} x_{tgw} = 3 & & \forall g \in G, t \in T \tag{C2}\\
& \text{Any 2 teams can be assinged to the same group once} & x_{tgw} + x_{t'gw} + x_{tg'w'} + x_{t'g'w'}  \leq 3 & & \forall t\neq t' \in T, g , g' \in G , w\neq w' \in W \tag{C3} \\
& \text{Each group has 1 hosting location} & \sum_{l} y_{lgw} = 1 & & \forall g \in G, w \in W \tag{C4}\\
& \text{group host location belongs to a Teams of the pool} & \sum_{p} x_{tgw} * M_{tl} \geq y_{lgw} & & \forall l \in L, g \in G, w \in W \tag{C5} \\
& \text{Each team needs to organise at least 1 tournament} & \sum_{lgw} y_{lgw} * M_{tl} \geq 1 & & \forall t \in T \tag{C6} \\
& \text{Definition of $z$} &  \begin{cases}
                 &z_{ll'w} \leq \sum_{g} y_{lgw}    \\
                 &z_{ll'w} \leq \sum_{g} y_{l'g'w+1} \tag{C7} \\
                 &z_{ll'w} \geq \sum_{g} y_{lgw} + \sum_{g'} y_{l'g'w+1} - 1
    \end{cases} &
\end{align}

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px

In [2]:
import cufflinks as cf
cf.go_offline()

In [3]:
# load the data

df_teams = pd.read_csv('data/Teams_Input.csv',sep=';',index_col=0)
df_teams

,Country,Location
Team_1,Portugal,Lisbon
Team_2,Mongolia,Ulaanbaatar
Team_3,Belgium,Brussels
Team_4,Iceland,Reykjavik
Team_5,Seychelles,Victoria
Team_6,United Kingdom,London
Team_7,Guinea,Conakry
Team_8,Eswatini,Mbabane
Team_9,Benin,Porto-Novo


In [4]:
team_location_map = df_teams.set_index('Country')['Location'].to_dict()
team_location_map

{'Portugal': 'Lisbon',
 'Mongolia': 'Ulaanbaatar',
 'Belgium': 'Brussels',
 'Iceland': 'Reykjavik',
 'Seychelles': 'Victoria',
 'United Kingdom': 'London',
 'Guinea': 'Conakry',
 'Eswatini': 'Mbabane',
 'Benin': 'Porto-Novo'}

In [5]:
df_fans = pd.read_csv('data/Fans_forecast.csv',sep=';',index_col=0)

In [6]:
df_fans.head()

,Weekend_1,Weekend_2,Weekend_3,Weekend_4
Lisbon,5000,12000,2500,11000
Ulaanbaatar,7000,12500,6500,15000
Brussels,16000,3000,7500,11500
Reykjavik,10000,4000,12500,10000
Victoria,15500,9500,4000,12000


In [7]:
# setting dimensions

Teams = list(df_teams['Country'])

Locations = df_teams['Location']

Weekends = list(range(1,len(df_fans.columns)+1))
print(Teams)
print(Weekends)

['Portugal', 'Mongolia', 'Belgium', 'Iceland', 'Seychelles', 'United Kingdom', 'Guinea', 'Eswatini', 'Benin']
[1, 2, 3, 4]


In [8]:
N_Teams = len(Teams)
N_Weekends = len(Weekends)

# numb of groups each Turn : Tournament sctructural value
GroupsXTurn = 3

# numb of teams in each group
TeamsXGroup = N_Teams/GroupsXTurn

In [9]:
# getting distance matrix (Km)
df_dist = pd.read_csv('data/Distances.csv',sep=';',index_col=0)

In [10]:
df_dist.head()

,Lisbon,Ulaanbaatar,Brussels,Reykjavik,Victoria,London,Conakry,Mbabane,Porto-Novo
Location,,,,,,,,,
Lisbon,0.000000,8521.933170,1715.157554,2960.706246,8172.888127,1586.542513,3042.357442,8331.657474,3758.839534
Ulaanbaatar,8521.933170,0.000000,6812.523638,6792.852993,7675.153119,6993.217230,11303.272710,11153.788000,10528.603980
Brussels,1715.157554,6812.523638,0.000000,2133.756729,7833.727289,322.849447,4688.429454,8944.363016,4918.694200
Reykjavik,2960.706246,6792.852993,2133.756729,0.000000,9851.919970,1894.445055,5835.492531,11035.887420,6696.592607
Victoria,8172.888127,7675.153119,7833.727289,9851.919970,0.000000,8138.251124,8070.099455,3530.486278,5997.955891


In [11]:
import pyomo.environ
from pyomo.environ import * 
#from pyomo.opt import SolverFactory, IOptSolver
import pyomo.core as pyomo

In [12]:
## create the model

In [13]:
model = ConcreteModel()
model.name = 'Model_PartIII'

In [14]:
# init dimensions

### Sets


\begin{align}
& \text{Locations}  &L &  & & \forall l \in L \\
& \text{Teams}  &T &  & &\forall t \in T \\
& \text{Weekend} &W  &  & &\forall w \in W\\
&\text{Groups}  &G  &  & &\forall g \in G\\  
\end{align}

In [15]:
# Players = Locations
model.T_dim = Set(initialize=Teams)

# Numbers of Locations initialization
model.L_dim = Set(initialize=Locations)

# Numbers of Turns initialization
model.W_dim = Set(initialize=Weekends)

# Numbers of Groups initialization
model.G_dim = Set(initialize=range( GroupsXTurn))


### Parameters
\begin{align}
& \text{Distance matrix} & D_{ll'} & &  & & \forall l,l' \in L\\
& \text{Expected fans per Weekend in each location} & F_{lw}  & &  & & \forall l \in L, w \in W\\
& \text{Team <> Location map} & M_{tl}  & &  & & \forall t \in T, l \in L\\
&  \text{Weight parameter} & P    & & & &
\end{align}


In [16]:
model.P_param = Param(mutable=True)

In [17]:
model.Dist_param = Param(model.L_dim, model.L_dim, initialize=lambda model, l1, l2: df_dist.loc[l1,l2])

model.F_param = Param(model.L_dim, model.W_dim, initialize=lambda model, l, w: df_fans.loc[l,'Weekend_%d' % w])

In [18]:
model.M_param = Param(model.T_dim, model.L_dim, initialize=lambda model, t, l: 1 if l == team_location_map[t] else 0 )

### Variables
\begin{align}
& \text{Team playing in pool (g,w)} & x_{tgw} & \in \{0,1\} &  & & \forall t \in T, g \in G,w \in W & \tag{V1} \\
& \text{Location hosting pool (g,w)}  & y_{lgw} & \in \{0,1\} & & & \forall l \in L, g \in G,w \in W & \tag{V2} \\
& & & & & & &\\
\end{align}

In [19]:
model.x_var = Var(model.T_dim, model.G_dim,  model.W_dim , domain=Binary)

In [20]:
model.y_var = Var( model.L_dim, model.G_dim, model.W_dim, domain=Binary)

\begin{align}
 & \text{Flight from $l$ to $l'$ between $w$ and $w+1$} & z_{ll'w} & \in \{0,1\}&  & &\forall l,l' \in L, w \in w & \tag{V3} \\
          & & & = \sum_{g} y_{lgw} * \sum_{g'} y_{l'g'w+1}  & & &  & 
\end{align}

In [21]:
model.z_var = Var(model.L_dim, model.L_dim, model.W_dim , domain=Binary)



\begin{align}
& \text{ } & \text{Total Travelled distance} & & E_T  & = & \sum_{ll'w} D_{ll'} * z_{ll'w} &  \\
& \text{ } & \text{Total Attending Fans} &  & E_F  & = & \sum_{glw} F_{lw} * y_{lgw} & \\
\end{align}

In [22]:
def e_travels(model):
    return sum(model.Dist_param[l,lp] * model.z_var[l,lp,w] \
               for l in model.L_dim for lp in model.L_dim for w in model.W_dim 
              )

In [23]:
def e_fans(model):
        return sum( model.F_param[l,w] * model.y_var[l,g,w] \
                       for g in model.G_dim for l in model.L_dim for w in model.W_dim                     )

In [24]:
model.e_travels = Expression( rule=e_travels)
model.e_fans = Expression( rule=e_fans)

### Objective
\begin{align}
 & & & &  \min_{zy}   E_T - W * E_F & \tag{obj}
\end{align}

In [25]:
def obj_rule(model):
         return  model.e_travels - model.P_param * model.e_fans

model.obj = Objective(rule=obj_rule,sense=minimize)

### Constraints

\begin{align}
& \text{Each player plays in a group each turn} & \sum_{g} x_{tgw} = 1 & & \forall t \in T, w \in W \tag{C1}\\
\end{align}

In [26]:
def rul1 (model,t,w):
    return sum(model.x_var[t,g,w] for g in model.G_dim ) == 1

model.rul1 = Constraint(model.T_dim, model.W_dim, rule=rul1,doc='constraint1')


\begin{align}
&\text{Each group consists of 3 teams each Turn} & \sum_{p} x_{tgw} = 3 & & \forall g \in G, t \in T \tag{C2}\\
\end{align}

In [27]:
def rul2 (model,g,w):
    return sum(model.x_var[t,g,w] for t in model.T_dim ) == 3

model.rul2 = Constraint(model.G_dim, model.W_dim, rule=rul2,doc='constraint2')


\begin{align}
&\text{Any 2 teams can be assinged to the same group once} & x_{tgw} + x_{t'gw} + x_{tg'w'} + x_{t'g'w'}  \leq 3 & & \forall t\neq t' \in T, g , g' \in G , w\neq w' \in W \tag{C3} \\
\end{align}

In [28]:

def rul3 (model,t,tp,g,gp,w,wp):
    if (t!=tp and w!=wp): # and g!=gp
        return (model.x_var[t,g,w] + model.x_var[tp,g,w]  + model.x_var[t,gp,wp]  + model.x_var[tp,gp,wp])<=3 
    else:
        return Constraint.Skip

model.rul3 = Constraint(model.T_dim, model.T_dim,model.G_dim,model.G_dim, model.W_dim,model.W_dim,
                        rule=rul3,doc='constraint3')


\begin{align}
& \text{Each group has 1 hosting location} & \sum_{l} y_{lgw} = 1 & & \forall g \in G, w \in W \tag{C4}\\
\end{align}

In [29]:
def rul4 (model,g,w):
    return sum(model.y_var[l,g,w] for l in model.L_dim ) == 1

model.rul4 = Constraint(model.G_dim, model.W_dim, rule=rul4,doc='constraint4')



\begin{align}
& \text{group host location belongs to a Teams of the pool} & \sum_{p} x_{tgw} * M_{tl} \geq y_{lgw} & & \forall l \in L, g \in G, w \in W \tag{C5} \\
\end{align}

In [30]:
#
def rules_5(model,l,g,w):

    return sum(model.x_var[t,g,w] * model.M_param[t,l]
               for t in model.T_dim ) >= model.y_var[l,g,w]


model.rul5 = Constraint(model.L_dim,model.G_dim,model.W_dim, rule=rules_5,doc='constraint5')


\begin{align}
& \text{Each team needs to organise at least 1 tournament} & \sum_{lgw} y_{lgw} * M_{tl} \geq 1 & & \forall t \in T \tag{C6} \\
\end{align}

In [31]:
def rules_loc(model,t):
        return sum(model.y_var[l,g,w] * model.M_param[t,l] for l in model.L_dim for g in model.G_dim for w in model.W_dim) >= 1


model.rulloc = Constraint(model.T_dim, rule=rules_loc,doc='constraint_loc')

\begin{align}
& \text{Definition of $z$} &  \begin{cases}
                 &z_{ll'w} \leq \sum_{g} y_{lgw}    \\
                 &z_{ll'w} \leq \sum_{g} y_{l'g'w+1} \tag{C7} \\
                 &z_{ll'w} \geq \sum_{g} y_{lgw} + \sum_{g'} y_{l'g'w+1} - 1
    \end{cases} &
\end{align}

In [32]:
# def of Z = Y_w * Y_w+1
def rules_Za(model,l,lp,w):
    if w <N_Weekends:
        return model.z_var[l,lp,w] <= sum(model.y_var[l,g,w] for g in model.G_dim)
    else:
        return model.z_var[l,lp,w] == 0


model.Za = Constraint(model.L_dim,model.L_dim,model.W_dim,
                         rule=rules_Za,doc='constraint_Za')

# def of Z = Y_w * Y_w+1
def rules_Zb(model,l,lp,w):
    if w <N_Weekends:
        return model.z_var[l,lp,w] <= sum(model.y_var[lp,gp,w+1] for gp in model.G_dim)
    else:
        return model.z_var[l,lp,w] == 0


model.Zb = Constraint(model.L_dim,model.L_dim,model.W_dim,
                         rule=rules_Zb,doc='constraint_Zb')

# def of Z = Y_w * Y_w+1

def rules_Zc(model,l,lp,w):
    if w <N_Weekends:
        return model.z_var[l,lp,w] >= sum(model.y_var[l,g,w] + model.y_var[lp,g,w+1] for g in model.G_dim) - 1
    else:
        return model.z_var[l,lp,w] == 0


model.Zc = Constraint(model.L_dim,model.L_dim,model.W_dim,
                         rule=rules_Zc,doc='constraint_Zc')

    

### Digression: Linearization of a product of Boolean Variables

Given
\begin{cases}
x & &\in \{0,1\}\\
y & &\in \{0,1\}\\
\end{cases}

we define
$z \equiv x * y \in \{0,1\}$ st
\begin{cases}
&z \leq x    \\
                 &z \leq y  \\
                 &z \geq x + y - 1 
\end{cases}


#### Tables of cases for $z$ 

|  ($x$,$y$) 	 | \| |$z$ |  
|---	 | ---|---	  |
|**(0,0)**   | \| |   0|  
|**(0,1)** |\| |   0|  
|**(1,0)** |\| |   0|   
|**(1,1)** |\| |   1|   




In [33]:
#opt = SolverFactory("glpk")
opt = SolverFactory("gurobi")

In [34]:
p_list = [0.1,0.2,0.5,1,2,5,10,15,20,25]
#p_list = [25]

In [35]:
models = {}
for p in p_list:
    print('-'*100)
    print('Doing %.1f' % p)
    models[p] = model.clone()
    models[p].name = 'Model_%f.1f' % p
    models[p].P_param = p

    opt.solve(models[p],tee=True)

----------------------------------------------------------------------------------------------------
Doing 0.1

--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmpj_5kfmxc.pyomo.lp
Reading time = 0.06 seconds
x541: 8926 rows, 541 columns, 35641 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 8926 rows, 541 columns and 35641 nonzeros
Model fingerprint: 0xc88d4c28
Variable types: 1 continuous, 540 integer (540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 6076 rows and 82 columns
Presolve time: 0.06s
Presolved: 2850 rows, 459 columns, 12069 nonzeros
Variable types: 0 continuous, 459 integer (459 binary)
Found heuristic solution: objective 143031.71239


  7528  1825 55642.9274   23  104 90862.2149 48446.1674  46.7%  63.3   35s
  9253  1984 80994.2138   29   91 90862.2149 53091.8338  41.6%  61.7   40s
 11477  2058     cutoff   27      90862.2149 57360.7582  36.9%  60.1   45s
 13437  2063 76973.4838   33  108 90862.2149 61113.5045  32.7%  58.9   50s
 14907  1998 72633.5174   24  106 90862.2149 64229.9991  29.3%  58.1   55s
 17185  1643     cutoff   29      90862.2149 69343.7735  23.7%  57.4   60s
 19983   844 81930.4034   33  100 90862.2149 76529.6471  15.8%  56.5   65s

Cutting planes:
  MIR: 24
  StrongCG: 4
  Zero half: 11

Explored 21995 nodes (1213138 simplex iterations) in 68.14 seconds
Thread count was 4 (of 4 available processors)

Solution count 10: 90862.2 90907.8 96655.4 ... 130336

Optimal solution found (tolerance 1.00e-04)
Best objective 9.086221492340e+04, best bound 9.086221492340e+04, gap 0.0000%
----------------------------------------------------------------------------------------------------
Doing 0.2

-------------

     0     0 -22333.918    0  152 112454.497 -22333.918   120%     -    2s
     0     0 -22333.887    0  153 112454.497 -22333.887   120%     -    2s
     0     0 -22333.887    0  157 112454.497 -22333.887   120%     -    2s
     0     0 -22333.887    0  154 112454.497 -22333.887   120%     -    2s
     0     2 -22328.943    0  151 112454.497 -22328.943   120%     -    2s
*   71    71              26    102491.48470 -21875.624   121%   144    3s
*   82    77              31    102284.87233 -21875.624   121%   130    3s
*   94    87              33    98032.559753 -21875.624   122%   124    3s
H  165   134                    94144.255803 -21265.113   123%   122    4s
H  188   140                    91129.235198 -21265.113   123%   113    4s
   258   204 22848.3362   11  116 91129.2352 -15740.689   117%   121    5s
*  589   395              36    88523.471407 -2323.0557   103%   105    7s
*  708   443              29    87056.730790 -1474.5298   102%  97.5    7s
   819   500 -1399.0713  

     0     0 -59480.151    0  124 61999.9516 -59480.151   196%     -    1s
     0     0 -59450.736    0  125 61999.9516 -59450.736   196%     -    1s
     0     0 -59448.603    0  128 61999.9516 -59448.603   196%     -    1s
     0     0 -59448.603    0  128 61999.9516 -59448.603   196%     -    1s
     0     0 -59404.373    0  133 61999.9516 -59404.373   196%     -    1s
     0     0 -59395.927    0  138 61999.9516 -59395.927   196%     -    1s
     0     0 -59395.703    0  138 61999.9516 -59395.703   196%     -    1s
     0     0 -59360.359    0  130 61999.9516 -59360.359   196%     -    1s
     0     0 -59360.349    0  141 61999.9516 -59360.349   196%     -    1s
     0     0 -59257.182    0  143 61999.9516 -59257.182   196%     -    2s
     0     0 -59176.694    0  138 61999.9516 -59176.694   195%     -    2s
     0     0 -59153.544    0  135 61999.9516 -59153.544   195%     -    2s
     0     0 -59149.073    0  137 61999.9516 -59149.073   195%     -    2s
     0     0 -59145.540  

     0     0 -128774.18    0  138 -20357.784 -128774.18   533%     -    1s
     0     0 -128774.18    0  140 -20357.784 -128774.18   533%     -    1s
     0     0 -128413.33    0  142 -20357.784 -128413.33   531%     -    1s
     0     0 -128368.28    0  138 -20357.784 -128368.28   531%     -    1s
     0     0 -128353.11    0  150 -20357.784 -128353.11   530%     -    1s
     0     0 -128342.65    0  146 -20357.784 -128342.65   530%     -    1s
     0     0 -128342.65    0  146 -20357.784 -128342.65   530%     -    1s
     0     0 -128084.95    0  139 -20357.784 -128084.95   529%     -    1s
     0     0 -128053.26    0  129 -20357.784 -128053.26   529%     -    1s
     0     0 -128025.02    0  137 -20357.784 -128025.02   529%     -    1s
     0     0 -128014.75    0  141 -20357.784 -128014.75   529%     -    1s
     0     0 -128007.34    0  141 -20357.784 -128007.34   529%     -    1s
     0     0 -128004.05    0  141 -20357.784 -128004.05   529%     -    1s
     0     0 -128003.84  


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -278500.00    0   24 -113468.29 -278500.00   145%     -    0s
H    0     0                    -128047.9506 -278500.00   117%     -    0s
     0     0 -276826.37    0  101 -128047.95 -276826.37   116%     -    0s
     0     0 -276773.58    0   96 -128047.95 -276773.58   116%     -    0s
     0     0 -274582.48    0   90 -128047.95 -274582.48   114%     -    0s
     0     0 -274522.47    0  100 -128047.95 -274522.47   114%     -    0s
     0     0 -262523.87    0  111 -128047.95 -262523.87   105%     -    0s
H    0     0                    -128767.9871 -262523.87   104%     -    0s
     0     0 -262381.73    0  114 -128767.99 -262381.73   104%     -    0s
     0     0 -262381.73    0  114 -128767.99 -262381.73   104%     -    0s
     0     0 -261111.10    0  124 -128767.99 -261111.10   103%     -    0s
H    0     0           

     0     0 -1519187.1    0   16 -1472675.4 -1519187.1  3.16%     -    0s
H    0     0                    -1519187.129 -1519187.1  0.00%     -    0s

Cutting planes:
  Gomory: 2
  Implied bound: 2
  Clique: 3
  MIR: 2
  Zero half: 2

Explored 1 nodes (1130 simplex iterations) in 0.34 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: -1.51919e+06 -1.47268e+06 -1.19347e+06 
No other solutions better than -1.51919e+06

Optimal solution found (tolerance 1.00e-04)
Best objective -1.519187129295e+06, best bound -1.519187129295e+06, gap 0.0000%
----------------------------------------------------------------------------------------------------
Doing 15.0

--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmpn2wq4k0c.pyomo.lp
Reading time = 0.05 seconds
x541: 8926 rows, 541 columns, 35641 nonzeros
Gurobi Opti

# Parse Results

In [36]:
class results:
    def __init__(self):
        self.df_final_calendar = pd.DataFrame()
        self.df_byTeam = pd.DataFrame()
        self.df_dist = pd.DataFrame()
        self.df_pos = pd.DataFrame()
        self.df_gkpi = pd.DataFrame()
        
    def parse_solution(self,_model):
        p = _model.P_param.value
        ## groups
        df_groups = v2df(_model.x_var,['Team','Group','Weekend'])
        df_groups = df_groups.reset_index().set_index(['Weekend','Group']).sort_index()

        ## locations
        df_loc = v2df(_model.y_var,['Location','Group','Weekend'])
        df_loc = df_loc.reset_index().set_index(['Weekend','Group']).sort_index()

        df_calendar = df_loc.merge(df_groups,left_index=True,right_index=True)

        df_cal_sf = df_calendar.reset_index().groupby(['Weekend','Group','Location']).agg(
            {"Team": {lambda l: l[0], lambda l: l[1],lambda l: l[2] } })
        df_cal_sf = df_cal_sf.rename({  '<lambda_%d>' % i :'Team_%d' % (i+1) for i in range(3)},axis=1)

        df_p = df_fans.T.copy()
        df_p['Weekend'] = df_p.index.map(lambda s: int(s.split("_")[-1]))
        df_p = df_p.melt(id_vars='Weekend',var_name='Location',value_name='Fans').set_index(['Weekend','Location'])
        df_c = df_cal_sf['Team']

        df_c = df_c.reset_index().set_index(['Weekend','Location'])

        df_final_calendar = df_c.merge(df_p,right_index=True,left_index=True)
        df_final_calendar = df_final_calendar.sort_index().drop('Group',axis=1)

        ordered_cols = ['Fans'] + list(df_final_calendar.columns)[:-1]
        df_final_calendar = df_final_calendar[ordered_cols]

        # reconciliation
        print('Reconcilation...')
        _df = df_calendar.reset_index().set_index('Team')
        _df = _df.pivot(columns='Weekend')['Location']

        for i in range(1,N_Weekends):
            _df['%d->%d' % (i,i+1)] = [ df_dist.loc[x] for x in zip(_df[i],_df[i+1])]

        df_byTeam = _df.copy()

        travel_cols = [ col for col in _df.columns if '->' in str(col) ]
        totdist = _df[ travel_cols ].sum(axis=1)

        tot_fans = df_final_calendar['Fans'].sum()

        model_obj = _model.obj.expr() 
        calc_obj = totdist.sum() - _model.P_param.value * tot_fans
        print("[%s]   %.2f vs %.2f" % ('OK' if  abs(model_obj - calc_obj)/calc_obj< 0.001 else 'NO'  ,model_obj,calc_obj))

        

        self.df_final_calendar = df_final_calendar
        self.df_byTeam = df_byTeam
        self.df_dist = df_byTeam[travel_cols]
        self.df_pos  = df_byTeam[[col for col in df_byTeam.columns if col not in travel_cols]]

        ## global KPIs
        kpis = {'Shortest Journey' : self.df_dist.sum(axis=1).min(),
               'Longest Journey' : self.df_dist.sum(axis=1).max(),
                'Delta' : self.df_dist.sum(axis=1).max() - self.df_dist.sum(axis=1).min(),
                'Mean Journey' : self.df_dist.sum(axis=1).mean(),
                'Total Travel' : self.df_dist.sum(axis=1).sum(),
               'Longest Leg' : self.df_dist.max().max(),
                'Lowest # Spectators' : self.df_final_calendar['Fans'].max(),
                'Highest # Spectators' : self.df_final_calendar['Fans'].min(),
                'Total # Spectators' : self.df_final_calendar['Fans'].sum(),
                'Mean # Spectators' : self.df_final_calendar['Fans'].mean()
               }

        df_g = pd.DataFrame.from_dict({p:kpis},orient='index')
        df_g.index.name = 'P'
        self.df_gkpi = df_g

        


def v2df(var,idxnames,getOn=True):
    rs = var.extract_values()
    df = pd.DataFrame(pd.Series(rs),columns=['value'])
    df.index.names = idxnames
    if getOn:
        df = df[df['value']>0].dropna()
        df = df.drop('value',axis=1)
    return df

# Example:

In [64]:
_result_25 = results()
_result_25.parse_solution(models[25])

Reconcilation...
[OK]   -3978469.83 vs -3978469.83


### Calendar

In [65]:
_result_25.df_final_calendar

Fans          Team_1          Team_2      Team_3
Weekend Location                                                      
1       Brussels     16000         Belgium          Guinea    Mongolia
        Porto-Novo   13500  United Kingdom           Benin     Iceland
        Victoria     15500      Seychelles        Eswatini    Portugal
2       Conakry      16500          Guinea           Benin  Seychelles
        Lisbon       12000        Mongolia         Iceland    Portugal
        Mbabane      12500  United Kingdom        Eswatini     Belgium
3       Brussels      7500         Belgium           Benin    Portugal
        London       15500      Seychelles  United Kingdom    Mongolia
        Reykjavik    12500          Guinea        Eswatini     Iceland
4       Brussels     11500         Iceland      Seychelles     Belgium
        London       16000  United Kingdom          Guinea    Portugal
        Ulaanbaatar  15000        Eswatini           Benin    Mongolia

In [66]:
_result_25.df_dist.iplot(kind='bar', barmode='stack', title='Travelled Distance per Player',);

### Position of each Team

In [67]:
_result_25.df_pos

Weekend,1,2,3,4
Team,,,,
Belgium,Brussels,Mbabane,Brussels,Brussels
Benin,Porto-Novo,Conakry,Brussels,Ulaanbaatar
Eswatini,Victoria,Mbabane,Reykjavik,Ulaanbaatar
Guinea,Brussels,Conakry,Reykjavik,London
Iceland,Porto-Novo,Lisbon,Reykjavik,Brussels
Mongolia,Brussels,Lisbon,London,Ulaanbaatar
Portugal,Victoria,Lisbon,Brussels,London
Seychelles,Victoria,Conakry,London,Brussels
United Kingdom,Porto-Novo,Mbabane,London,London


# Data Viz

In [68]:
df_geodata = pd.read_csv('data/GeoData.csv',sep=';',index_col=0)

In [69]:
df_geodata

,Country,Location,Address,GP_Loc,GeoPoint,Latitude,Longitude
Team_1,Portugal,Lisbon,Portugal Lisbon,"Lisbon, Grande Lisboa, Área Metropolitana de L...","38 42m 27.9025s N, 9 8m 11.7308s W",38.707751,-9.136592
Team_2,Mongolia,Ulaanbaatar,Mongolia Ulaanbaatar,"Ulaanbaatar, Mongolia","47 55m 6.48336s N, 106 55m 3.72576s E",47.918468,106.917702
Team_3,Belgium,Brussels,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437
Team_4,Iceland,Reykjavik,Iceland Reykjavik,"Reykjavik, Capital Region, Iceland","64 14m 40.2128s N, 21 46m 9.34886s W",64.244504,-21.769264
Team_5,Seychelles,Victoria,Seychelles Victoria,"Victoria, Mahé, SEZ, Seychelles","4 37m 23.5506s S, 55 27m 8.4924s E",-4.623208,55.452359
Team_6,United Kingdom,London,United Kingdom London,"London, Greater London, England, SW1A 2DX, Uni...","51 30m 26.3588s N, 0 7m 39.5306s W",51.507322,-0.127647
Team_7,Guinea,Conakry,Guinea Conakry,"Guinea-Conakry, Rua Osvaldo Vieira, Praça, Rei...","11 51m 36.8586s N, 15 35m 4.36956s W",11.860238,-15.584547
Team_8,Eswatini,Mbabane,Eswatini Mbabane,"Mbabane, Inkhundla Mbabane, Hhohho, H100, Swaz...","26 19m 32.682s S, 31 8m 40.7868s E",-26.325745,31.144663
Team_9,Benin,Porto-Novo,Benin Porto-Novo,"Porto-Novo, Porto Novo, Ouémé, 03BP647, Benin","6 29m 56.6585s N, 2 37m 31.21s E",6.499072,2.625336


In [70]:
px.scatter_geo(df_geodata,lat='Latitude',lon='Longitude',hover_name='Location',hover_data = ['Country'],
              size=[6 for i in df_geodata.index],size_max=10,opacity=0.95)

In [71]:
df_cal_25 = _result_25.df_final_calendar

In [72]:
df_cal_25_pp = df_cal_25.reset_index().merge(df_geodata,on=['Location'])
df_cal_25_pp['Weekend_'] = df_cal_25_pp['Weekend'].apply(str)
df_cal_25_pp.head()

,Weekend,Location,Fans,Team_1,Team_2,Team_3,Country,Address,GP_Loc,GeoPoint,Latitude,Longitude,Weekend_
0,1,Brussels,16000,Belgium,Guinea,Mongolia,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437,1
1,3,Brussels,7500,Belgium,Benin,Portugal,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437,3
2,4,Brussels,11500,Iceland,Seychelles,Belgium,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437,4
3,1,Porto-Novo,13500,United Kingdom,Benin,Iceland,Benin,Benin Porto-Novo,"Porto-Novo, Porto Novo, Ouémé, 03BP647, Benin","6 29m 56.6585s N, 2 37m 31.21s E",6.499072,2.625336,1
4,1,Victoria,15500,Seychelles,Eswatini,Portugal,Seychelles,Seychelles Victoria,"Victoria, Mahé, SEZ, Seychelles","4 37m 23.5506s S, 55 27m 8.4924s E",-4.623208,55.452359,1


In [73]:
px.scatter_geo(df_cal_25_pp,lat='Latitude',lon='Longitude',
               color='Weekend_',
               hover_name='Location',hover_data = ['Team_1','Team_2','Team_3'],
              opacity=.95,size=[6 for i in df_cal_pp.index],size_max=10)

In [74]:
df_pos_25 = _result_25.df_pos.reset_index().melt(id_vars='Team',value_name='Location')
df_pos_25.head()

,Team,Weekend,Location
0,Belgium,1,Brussels
1,Benin,1,Porto-Novo
2,Eswatini,1,Victoria
3,Guinea,1,Brussels
4,Iceland,1,Porto-Novo


In [75]:
df_aug_25 = df_pos_25.merge(df_geodata,on=['Location'])

In [76]:
df_aug_25.head()

,Team,Weekend,Location,Country,Address,GP_Loc,GeoPoint,Latitude,Longitude
0,Belgium,1,Brussels,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437
1,Guinea,1,Brussels,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437
2,Mongolia,1,Brussels,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437
3,Belgium,3,Brussels,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437
4,Benin,3,Brussels,Belgium,Belgium Brussels,"Brussels, Ville de Bruxelles - Stad Brussel, B...","50 50m 37.2152s N, 4 22m 2.7721s E",50.843671,4.367437


In [77]:
print(_result_25.df_gkpi['Total Travel'])

P
25    121530.169904
Name: Total Travel, dtype: float64


In [78]:
px.line_geo(df_aug_25.sort_values('Weekend'),lat='Latitude',lon='Longitude',
            color='Team',line_group='Team',
            hover_name='Team',animation_group='Weekend',hover_data=['Location','Weekend'],)


In [79]:
_result_01 = results()
_result_01.parse_solution(models[0.1])

df_pos_01 = _result_01.df_pos.reset_index().melt(id_vars='Team',value_name='Location')
df_aug_01 = df_pos_01.merge(df_geodata,on=['Location'])

Reconcilation...
[OK]   90862.21 vs 90862.21


In [80]:
print(_result_01.df_gkpi['Total Travel'])

P
0.1    104462.214923
Name: Total Travel, dtype: float64


In [81]:
px.line_geo(df_aug_01.sort_values('Weekend'),lat='Latitude',lon='Longitude',
            color='Team',line_group='Team',
            hover_name='Team',animation_group='Weekend',hover_data=['Location','Weekend'],)

# Analysis of Solutions

In [82]:
all_results = {}
for p in p_list:
    _res = results()
    _res.parse_solution(models[p])
    all_results[p] = _res

Reconcilation...
[OK]   90862.21 vs 90862.21
Reconcilation...
[OK]   77157.76 vs 77157.76
Reconcilation...
[OK]   34062.87 vs 34062.87
Reconcilation...
[OK]   -47687.13 vs -47687.13
Reconcilation...
[OK]   -211187.13 vs -211187.13
Reconcilation...
[OK]   -701687.13 vs -701687.13
Reconcilation...
[OK]   -1519187.13 vs -1519187.13
Reconcilation...
[OK]   -2338469.83 vs -2338469.83
Reconcilation...
[OK]   -3158469.83 vs -3158469.83
Reconcilation...
[OK]   -3978469.83 vs -3978469.83


In [83]:
all_results[25].df_pos

Weekend,1,2,3,4
Team,,,,
Belgium,Brussels,Mbabane,Brussels,Brussels
Benin,Porto-Novo,Conakry,Brussels,Ulaanbaatar
Eswatini,Victoria,Mbabane,Reykjavik,Ulaanbaatar
Guinea,Brussels,Conakry,Reykjavik,London
Iceland,Porto-Novo,Lisbon,Reykjavik,Brussels
Mongolia,Brussels,Lisbon,London,Ulaanbaatar
Portugal,Victoria,Lisbon,Brussels,London
Seychelles,Victoria,Conakry,London,Brussels
United Kingdom,Porto-Novo,Mbabane,London,London


# Global KPIs


In [84]:
df_gkpis = pd.DataFrame()
df_TeamAnalysis = pd.DataFrame()
for p,r in all_results.items():
    df_gkpis = df_gkpis.append(r.df_gkpi)
    _df = r.df_dist
    _df['Total Travel'] = _df.sum(axis=1)
    _df['P'] = p
    df_TeamAnalysis = df_TeamAnalysis.append(_df)

In [85]:
df_TeamAnalysis

Weekend,1->2,2->3,3->4,Total Travel,P
Team,,,,,
Belgium,7833.727289,0.000000,0.000000,7833.727289,0.1
Benin,2086.359544,4918.694200,6812.523638,13817.577382,0.1
Eswatini,4761.346862,6696.592607,2133.756729,13591.696198,0.1
Guinea,3042.357442,1586.542513,322.849447,4951.749402,0.1
Iceland,4688.429454,2133.756729,1894.445055,8716.631238,0.1
...,...,...,...,...,...
Iceland,3758.839534,2960.706246,2133.756729,8853.302509,25.0
Mongolia,1715.157554,1586.542513,6993.217230,10294.917297,25.0
Portugal,8172.888127,1715.157554,322.849447,10210.895128,25.0


In [86]:
df_gkpis

,Shortest Journey,Longest Journey,Delta,Mean Journey,Total Travel,Longest Leg,Lowest # Spectators,Highest # Spectators,Total # Spectators,Mean # Spectators
P,,,,,,,,,,
0.1,4951.749402,17926.447366,12974.697964,11606.912769,104462.214923,8944.363016,16000,2500,136000,11333.333333
0.2,4951.749402,17987.509111,13035.759709,11628.640158,104657.761421,9149.200410,16000,2500,137500,11458.333333
0.5,5080.364443,20518.233516,15437.869073,12868.096745,115812.870705,11035.887420,16500,7500,163500,13625.000000
1.0,4936.775844,22345.180037,17408.404193,12868.096745,115812.870705,11035.887420,16500,7500,163500,13625.000000
2.0,4951.749402,19672.903918,14721.154516,12868.096745,115812.870705,11035.887420,16500,7500,163500,13625.000000
5.0,5011.278901,20518.233516,15506.954615,12868.096745,115812.870705,11035.887420,16500,7500,163500,13625.000000
10.0,4936.775844,20903.764502,15966.988658,12868.096745,115812.870705,11035.887420,16500,7500,163500,13625.000000
15.0,3301.700067,19672.903918,16371.203851,13503.352212,121530.169904,11035.887420,16500,7500,164000,13666.666667
20.0,5668.231494,19672.903918,14004.672424,13503.352212,121530.169904,11035.887420,16500,7500,164000,13666.666667


In [87]:
df_gkpis.iplot(kind='scatter',x='Total # Spectators',y='Total Travel', mode='markers',
               xTitle='Total # Spectators',yTitle='Total Travelled Distance',)

In [88]:
px.scatter(df_gkpis.reset_index(),x='Total # Spectators',y='Total Travel',marginal_x="histogram",marginal_y="histogram",color='P')

## Pareto front

In [89]:
px.scatter_3d(df_gkpis.reset_index(),x='Total # Spectators',y='Total Travel',z='Delta',color='P',
             width=900, height=600)

## Tendency to Travel of partecipants

In [90]:
df_TeamAnalysis.pivot_table(index='P',columns='Team',values='Total Travel').iplot(kind='box',boxpoints='all',)

In [91]:
px.box(df_TeamAnalysis.reset_index(),x='Team',y='Total Travel',hover_data=['P'],points='all')

# Challenge: Gurobi VS GLPK

In [139]:
opt_glpk = SolverFactory("glpk")


In [140]:
p_test = 3

In [141]:
model_gu = model.clone()
model_gu.P_param = p_test



In [142]:
%%time
res_gu = opt.solve(model_gu,tee=True)


--------------------------------------------
--------------------------------------------

Using license file C:\Users\niccolomoretti\gurobi.lic
Read LP format model from file C:\Users\NICCOL~1\AppData\Local\Temp\tmpso0_2az5.pyomo.lp
Reading time = 0.07 seconds
x541: 8926 rows, 541 columns, 35641 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 8926 rows, 541 columns and 35641 nonzeros
Model fingerprint: 0x43d970bf
Variable types: 1 continuous, 540 integer (540 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 7e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 6076 rows and 82 columns
Presolve time: 0.08s
Presolved: 2850 rows, 459 columns, 12069 nonzeros
Variable types: 0 continuous, 459 integer (459 binary)
Found heuristic solution: objective -383468.2876

Root relaxation: objective -5.639977e+05, 534 iterations, 0.05 seconds

    Nodes    |    Current Node    |   

In [143]:
model_gl = model.clone()
model_gl.P_param = p_test

In [144]:
%%time
res_gl = opt_glpk.solve(model_gl,tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\NICCOL~1\AppData\Local\Temp\tmp6ble7f8b.glpk.raw --wglp
 C:\Users\NICCOL~1\AppData\Local\Temp\tmp_j5r5s7l.glpk.glp --cpxlp C:\Users\NICCOL~1\AppData\Local\Temp\tmp3st8igh8.pyomo.lp
Reading problem data from 'C:\Users\NICCOL~1\AppData\Local\Temp\tmp3st8igh8.pyomo.lp'...
C:\Users\NICCOL~1\AppData\Local\Temp\tmp3st8igh8.pyomo.lp:63364: warning: lower bound of variable 'x1' redefined
C:\Users\NICCOL~1\AppData\Local\Temp\tmp3st8igh8.pyomo.lp:63364: warning: upper bound of variable 'x1' redefined
8926 rows, 541 columns, 35641 non-zeros
540 integer variables, all of which are binary
63904 lines were read
Writing problem data to 'C:\Users\NICCOL~1\AppData\Local\Temp\tmp_j5r5s7l.glpk.glp'...
54191 lines were written
GLPK Integer Optimizer, v4.65
8926 rows, 541 columns, 35641 non-zeros
540 integer variables, all of which are binary
Preprocessing...
8262 hidden covering inequaliti(es) were detected
868

In [145]:
res_gu['Solver'][0]['Wall time']

'3.8766040802001953'

In [146]:
res_gl['Solver'][0]['Time']

27.598974466323853

In [151]:
model_gu.obj.expr()

-538187.1292954

In [152]:
model_gl.obj.expr()

-538187.1292954